# Inputting game stats
-note the data is 2016

In [1]:
import numpy as np

class GameStats(object):          
    
    def __init__(self, homeTeamNameIndex, homeTeamScoreIndex, homeTeamStatsIndex, visitorTeamNameIndex, visitorTeamScoreIndex, visitorTeamStatsIndex):
        #parse the text file
        self.statsFile = open("baseball2016.txt", "r")
        self.topArray = []
        self.sideArray = []  
        self.sc = np.zeros((30,30,30), np.int32) 
        self.sc[:,:,:] = -1  
        self.am = np.zeros((30,30), np.float32)
        self.gameList = []
        
        seenTeams = []
        for line in self.statsFile:
            token = line.split(',')  #tokenize the string
            tokenIndex = [homeTeamNameIndex, homeTeamScoreIndex, visitorTeamNameIndex, visitorTeamScoreIndex] + [i for i in homeTeamStatsIndex] + [i for i in visitorTeamStatsIndex]
            attributes = dict()
            
            for i in range(len(token)):
                if(i in tokenIndex):
                    attributes[i] = self.removeQuotes(token[i])
                        
            self.addScore(attributes[homeTeamNameIndex], attributes[visitorTeamNameIndex], attributes[homeTeamScoreIndex], attributes[visitorTeamScoreIndex])                
            self.addGame(attributes[homeTeamNameIndex], attributes[homeTeamScoreIndex], [attributes[i] for i in homeTeamStatsIndex], attributes[visitorTeamNameIndex], attributes[visitorTeamScoreIndex], [attributes[i] for i in visitorTeamStatsIndex])
            
            if(attributes[homeTeamNameIndex] not in seenTeams):
                seenTeams.append(attributes[homeTeamNameIndex])
            
        self.buildAvgMatrix()
        self.statsFile.close()
        #self.gameList = [bin(2**i)[2:].zfill(len(seenTeams)) if x == seenTeams[i] else x for i in range(len(seenTeams)) for x in self.gameList]
        # take the teams and convert the indexes of the order that they appeared in into the hot one format by rasiing
        # 2 to the power of them and then converting them into binary.
        seenTeamsDict = {k: v for v, k in enumerate(seenTeams)}
        temp = []
        for x in self.gameList:
            tempi = []
            for z in x:
                if(z in seenTeams):
                    tempi.append(bin(2**seenTeamsDict[z])[2:].zfill(len(seenTeamsDict)))
                else:
                    tempi.append(z)
            temp.append(tempi)
        self.gameList = temp
        
    def removeQuotes(self, string):
        if (string.startswith('"') and string.endswith('"')) or (string.startswith("'") and string.endswith("'")):
            return string[1:-1]
        return string  
    
    def addGame(self, team1, score1, stats1, team2, score2, stats2):
        self.gameList.append([team1, score1, stats1, team2, score2, stats2])
        
    #give it two teams, the scores, and it will add it to the matrix
    def addScore(self, team1, team2, score1, score2):
        '''
        for a team in top array, the index in the array corrisponds to the matrix column there located in
        for a team in side array, the index in the array corrisponds to the matrix row there located in
        '''
        #team 1 score entry
        try:
            row = self.sideArray.index(team2)    

        except:
            self.sideArray.append(team2)
            row = self.sideArray.index(team2)    

        try:
            col = self.topArray.index(team1)
        except:
            self.topArray.append(team1)
            col = self.topArray.index(team1)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score1
                break
            counter += 1
        self.sc[row, col] = temp
        
        #team 2 score entry
        try:
            row = self.sideArray.index(team1)    
        except:
            self.sideArray.append(team1)
            row = self.sideArray.index(team1)    
            
        try:
            col = self.topArray.index(team2)
        except:
            self.topArray.append(team2)
            col = self.topArray.index(team2)
        temp = self.sc[row, col]
        counter = 0
        for e in temp:
            if (e == -1):
                temp[counter] = score2
                break
            counter += 1
        self.sc[row, col] = temp
    
    #returns the score(s) for match up
    def getScore(self, team1, team2, gameSelect = None):
        print(team1, team2)
        try:
            score1 = self.sc[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.sc[self.sideArray.index(team1), self.topArray.index(team2)]
            if (gameSelect == None):
                print(team1, score1)
                print(team2, score2)
            else:
                print(team1, score1[gameSelect])
                print(team2, score2[gameSelect])
        except:
            print('Invalid input of teams')
    
    def getGameList(self):
        return self.gameList
    
    #constructs a matrix of the avg score in a matchup
    def buildAvgMatrix(self): 
        for col in range(len(self.sc[:,0])):   #depth
            for row in range(len(self.sc[0, :])):  #width
                tempScore = self.sc[row, col]
                avgScore = 0.0
                count = 0.0
                for j in tempScore:
                    if (j != -1):
                        avgScore += j
                        count += 1
                    else:
                        break
                try:
                    avgScore = avgScore / count
                except:
                    avgScore = -1
                self.am[row, col] = avgScore
    
    #get the value of the avg score for a match up
    def getAvgScore(self, team1, team2):
        try:
            score1 = self.am[self.sideArray.index(team2), self.topArray.index(team1)]
            score2 = self.am[self.sideArray.index(team1), self.topArray.index(team2)]
            print(team1, score1)
            print(team2, score2)        
        except:
            print('Invalid input of teams')

In [2]:
import tensorflow as tf
import tflearn

tf.reset_default_graph()

# Parse the game files and grab the stats we want
# the indexes are put in the order: homeTeamName, homeTeamScore, homeTeamStats, awayName, awayScore, awayStats
stats = GameStats(3, 10, [5, 51, 55], 6, 9, [8, 23, 27])
gameList = stats.getGameList() # get the list of games
avgStats = dict()
np.set_printoptions(precision=4)  #IF you wanna remove this i recommend restarting the kernal


#ERROR in the data, scroll down to see the outlire binary number
for row in gameList:
    print(row[2], row[5])

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
(['1', '0', '0'], ['1', '1', '0'])
(['1', '1', '0'], ['1', '1', '0'])
(['1', '2', '1'], ['1', '0', '0'])
(['1', '1', '0'], ['1', '3', '0'])
(['1', '3', '0'], ['1', '3', '0'])
(['1', '2', '0'], ['1', '1', '0'])
(['2', '1', '0'], ['2', '1', '0'])
(['1', '0', '0'], ['1', '0', '0'])
(['1', '2', '0'], ['1', '4', '0'])
(['1', '0', '0'], ['1', '1', '0'])
(['1', '3', '0'], ['1', '1', '0'])
(['1', '0', '0'], ['1', '3', '0'])
(['1', '1', '0'], ['1', '7', '0'])
(['2', '1', '0'], ['2', '1', '0'])
(['1', '0', '0'], ['1', '1', '0'])
(['2', '1', '0'], ['2', '0', '0'])
(['1', '1', '0'], ['1', '2', '0'])
(['2', '3', '0'], ['2', '0', '0'])
(['3', '1', '0'], ['3', '0', '0'])
(['2', '2', '0'], ['2', '3', '0'])
(['2', '3', '0'], ['2', '2', '0'])
(['1', '4', '0'], ['1', '2', '1'])
(['2', '2', '1'], ['2', '1', '0'])
(['2', '1', '0'], ['2', '2', '2'])
(['2', '0', '0'], ['2', '2', '0'])
(['2', '3', '0'], ['2', '3', '1'

In [3]:
#getting rid of the strings
def removeQuotes(gameList):
    for row in gameList:
        for x in xrange(len(row)):
            #convert scores strings to float
            if (x == 1 or x == 4): row[x] = float(row[x])
            #convert arrays to floats
            if (x == 2 or x == 5): row[x] = map(float, row[x])
    return gameList

#adding up all the teams stats
gameList = removeQuotes(gameList)
for row in gameList:
    #visitors
    print("visitor", row[0])
    if avgStats.has_key(row[0]):
        print(avgStats[row[0]])
        avgStats[row[0]][0] = 0   #set the proir games to 0,because there set to right amount next line
        avgStats[row[0]] = np.sum([avgStats[row[0]], row[2]], axis=0)  #total stats + indivudal game stats
    else:
        avgStats[row[0]] = row[2]
    print(row[2])
    print(avgStats[row[0]])
    print('-------------')
    
    #home
    print("home", row[3])
    if avgStats.has_key(row[3]):
        
        print(avgStats[row[3]])
        avgStats[row[3]][0] = 0
        avgStats[row[3]] = np.sum([avgStats[row[3]], row[5]], axis=0)
    else:
        avgStats[row[3]] = row[5]
    print(row[5])
    print(avgStats[row[3]])
    print('-------------')




('visitor', '000000000000000000000000000001')
[1.0, 0.0, 0.0]
[1.0, 0.0, 0.0]
-------------
('home', '000001000000000000000000000000')
[1.0, 1.0, 0.0]
[1.0, 1.0, 0.0]
-------------
('visitor', '000000000000000000000000000010')
[1.0, 1.0, 0.0]
[1.0, 1.0, 0.0]
-------------
('home', '000000000000100000000000000000')
[1.0, 1.0, 0.0]
[1.0, 1.0, 0.0]
-------------
('visitor', '000000000000000000000000000100')
[1.0, 2.0, 1.0]
[1.0, 2.0, 1.0]
-------------
('home', '000000001000000000000000000000')
[1.0, 0.0, 0.0]
[1.0, 0.0, 0.0]
-------------
('visitor', '000000000000000000000000001000')
[1.0, 1.0, 0.0]
[1.0, 1.0, 0.0]
-------------
('home', '000010000000000000000000000000')
[1.0, 3.0, 0.0]
[1.0, 3.0, 0.0]
-------------
('visitor', '000000000000000000000000010000')
[1.0, 3.0, 0.0]
[1.0, 3.0, 0.0]
-------------
('home', '000000100000000000000000000000')
[1.0, 3.0, 0.0]
[1.0, 3.0, 0.0]
-------------
('visitor', '000000000000000000000000100000')
[1.0, 2.0, 0.0]
[1.0, 2.0, 0.0]
-------------
('h

In [4]:
#make the average..
keyList = avgStats.keys()    
for key in keyList:
    print(key)
    print(avgStats[key])  #print total stats
    for index in xrange(1, len(avgStats[key])):  #divide all stats by games played (162, 162)
        avgStats[key][index] = avgStats[key][index] / avgStats[key][0]  
    print(avgStats[key])  #print the avg stats
    print('-------------------------------------------')

000000000000000000000000001000
[ 162.  205.   31.]
[ 162.        1.2654    0.1914]
-------------------------------------------
000000000000000100000000000000
[ 161.  250.   26.]
[ 161.        1.5528    0.1615]
-------------------------------------------
000000000000000010000000000000
[ 162.  320.   18.]
[  1.6200e+02   1.9753e+00   1.1111e-01]
-------------------------------------------
000000000000000001000000000000
[ 162.  251.   24.]
[  1.6200e+02   1.5494e+00   1.4815e-01]
-------------------------------------------
000000000000001000000000000000
[ 162.  260.   19.]
[  1.6200e+02   1.6049e+00   1.1728e-01]
-------------------------------------------
000000000000000000010000000000
[ 162.  262.   41.]
[ 162.        1.6173    0.2531]
-------------------------------------------
000010000000000000000000000000
[ 162.  275.   22.]
[  1.6200e+02   1.6975e+00   1.3580e-01]
-------------------------------------------
000000000000000000000100000000
[ 162.  252.   52.]
[ 162.        1.5556    

In [5]:
#needs to be imported again
stats = GameStats(3, 10, [5, 51, 55], 6, 9, [8, 23, 27])
gameList = stats.getGameList() # get the list of games

#fixing 000000000000000000000000100000 because the hits number is bad
avgStats['000000000000000000000000100000'][1] = 1.59   #some arbitrary value


#adding avg values to the list
for row in gameList:
    #visitor
    temp = avgStats[row[0]]   #using key to get avg stats array
    temp[0] = row[2][0]  #perserve the game num in the swap
    temp = np.around(temp, decimals=4)
    row[2] = temp
    #home
    temp = avgStats[row[3]]   #using key to get avg stats array
    temp[0] = row[5][0]
    temp = np.around(temp, decimals=4)
    row[5] = temp 

In [6]:
#the difference
gameList = removeQuotes(gameList)   #because i initialized gamestats above, must remove quote again
diffList = np.array([])


for row in gameList:
    diff = np.array([])    
    diff =  np.subtract(row[2], row[5])   #visistor stats - home stats (not including score)
    diff = np.append(diff, np.subtract(row[1], row[4]))   #visitor score - home score
    diff = np.delete(diff, 0)      #delete the game played column because subtracting those = 0
    
    diffList = np.append(diffList, diff)
    print(diff)

    


[ 0.0617  0.1543  1.    ]
[ 0.2222 -0.1235 -2.    ]
[-0.2717 -0.1605  3.    ]
[-0.4321  0.0556 -9.    ]
[ 0.1049  0.0556  1.    ]
[-0.1199  0.0556 -1.    ]
[ 0.2222 -0.1235 -2.    ]
[ 0.0803  0.0247  1.    ]
[-0.1543  0.0432 -5.    ]
[-0.0779 -0.002  -1.    ]
[ 0.2901 -0.0308  4.    ]
[ -1.9130e-01   6.2000e-03  -9.0000e+00]
[ -0.3086   0.0493 -15.    ]
[-0.4321  0.0556 -5.    ]
[-0.2332 -0.0196 -4.    ]
[ 0.0617  0.1543 -2.    ]
[ 0.2531 -0.0185 -2.    ]
[-0.1199  0.0556 -1.    ]
[ 0.2222 -0.1235  1.    ]
[ 0.0803  0.0247 -8.    ]
[-0.1543  0.0432  5.    ]
[-0.0373 -0.1366 -1.    ]
[-0.1913  0.0062 -1.    ]
[-0.2717 -0.1605  1.    ]
[-0.3086  0.0493 -3.    ]
[ 0.1049  0.0556  2.    ]
[-0.2332 -0.0196  1.    ]
[  0.2531  -0.0185  10.    ]
[-0.1199  0.0556  1.    ]
[ 0.2222 -0.1235  2.    ]
[ 0.0803  0.0247 -4.    ]
[-0.1543  0.0432 -1.    ]
[-0.0779 -0.002  -2.    ]
[ 0.2901 -0.0308  1.    ]
[-0.0373 -0.1366 -4.    ]
[-0.1913  0.0062  1.    ]
[-0.2717 -0.1605  4.    ]
[-0.3086  0.0493 